<h1>Import Relevant Libraries and Load Raw Collected Tweets</h1>

In [18]:
#nltk.download('wordnet')
#nltk.download('wordnet')
#nltk.download('stopwords')

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import nltk

from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer


from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer() 


#from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
#analyser = SentimentIntensityAnalyzer()

dataset = pd.read_csv('politicaltweetsmany.csv')
corpus = []
sentiments= []


<h1>Preprocess Tweets,remove noise and create a Corpus</h1>

In [19]:
y=[]
flag=0
for i in range(0,75001):
    tweet=dataset['text'][i]
    #print(tweet)
    
    tweet = re.sub(r"http\S+", "", tweet)
   # print(tweet)
#removing  emojis and n
    tweet = re.sub(r"#[a-zA-Z0-9]+", "", tweet)
    tweet = re.sub(r"\\x[a-zA-Z0-9]+", "", tweet)
    tweet = re.sub(r"\\n", " ", tweet)
 
    
    tweet = re.sub('[^a-zA-Z]', ' ', tweet)
    
    
    tweet = re.sub(r"\W"," ",tweet)
    
    tweet = re.sub(r"\d"," ",tweet)
    
    tweet = re.sub(r"s+[a-z]\s+"," ",tweet)
    
    tweet = re.sub(r"\s+[a-z]$"," ",tweet)
    
    tweet = re.sub(r"^[a-z]\s+"," ",tweet)
    
    tweet = re.sub(r"\s+"," ",tweet)
    
    tweet = tweet.lower()
    tweet = tweet.split()
    #Stemming 
    ps = PorterStemmer()
    tweet = [ps.stem(word) for word in tweet if not word in set(stopwords.words('english'))]
    #tweet = [lemmatizer.lemmatize(word) for word in tweet if not word in set(stopwords.words('english'))]
    #print(tweet)
    tweet = ' '.join(tweet)
    
    score = TextBlob(tweet)
    #compound=score['compound']
    #print("score=",score)
   # print(a.sentiment.polarity)
    sent=0
    if(score.sentiment.polarity<0):
        sent=0
    elif(score.sentiment.polarity>0):
        sent=1
    
    
        
    if(score.sentiment.polarity!=0):
        
        sentiments.append(sent)
    #print(dataset['text'][i],"sentiment=",sent)      
        corpus.append(tweet)
        y.append(sent)
    
    




<h1>Feature Extraction and Training Different models to find the best model </h1>

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
cv = TfidfVectorizer(max_features=700)
X = cv.fit_transform(corpus).toarray()
#y = datasset.iloc[:, 1].values

#splitting the dtaset into the trning set and testset
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

<h1>Naive Bayes</h1>

In [20]:
#fitting Naivebayes to the training set
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

#predicting the test set results
y_pred = classifier.predict(X_test)

#mking confusion matrix
from sklearn.metrics import confusion_matrix
cm =  confusion_matrix(y_test, y_pred)
print(cm)
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_pred))

/home/vedant/anaconda3/lib/python3.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


[[1375  240]
 [1010 4840]]
0.8325519089082385


<h1>Decision Tree</h1>

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
cv = TfidfVectorizer(max_features=2000)
X = cv.fit_transform(corpus).toarray()
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)
from sklearn.tree import DecisionTreeClassifier
model=DecisionTreeClassifier()
model.fit(X_train, y_train)

#predicting the test set results
y_pred = model.predict(X_test)

#mking confusion matrix
from sklearn.metrics import confusion_matrix
cm =  confusion_matrix(y_test, y_pred)
print(cm)
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_pred))



[[1459  156]
 [ 190 5660]]
0.9536503683858004


<h1>SVM</h1>

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
cv = TfidfVectorizer(max_features=700)
X = cv.fit_transform(corpus).toarray()
#y = datasset.iloc[:, 1].values

#splitting the dtaset into the trning set and testset
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)



from sklearn.svm import SVC  
svclassifier = SVC(kernel='linear')  
svclassifier.fit(X_train, y_train)  
y_pred = svclassifier.predict(X_test)  
from sklearn.metrics import confusion_matrix
cm =  confusion_matrix(y_test, y_pred)
print(cm)
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_pred))


[[1265  350]
 [ 124 5726]]
0.936503683858004


In [22]:
p=0
n=0
for i in y_train:
    if(i==1):
        p=p+1
    elif(i==0):
        n=n+1
print("p=",p,"n=",n)       
p=0
n=0
for i in y_test:
    if(i==1):
        p=p+1
    elif(i==0):
        n=n+1
print("p=",p,"n=",n)       

p=0
n=0
for i in y_pred:
    if(i==1):
        p=p+1
    elif(i==0):
        n=n+1
print("p=",p,"n=",n)       

p= 23376 n= 6480
p= 5850 n= 1615
p= 5816 n= 1649


<h1>Store Preprocessed tweets in File for Future Use</h1>

In [24]:
import csv
import itertools
csvFile = open('preprocessedtweets2.csv', 'a')
#Use csv Writer
csvWriter = csv.writer(csvFile)
csvWriter.writerow(["tweet","sentiment"])
# minimum among all the three  
for (tweet, sentiment) in zip(corpus,y): 
     csvWriter.writerow([tweet,sentiment])

<h1>Random Forest</h1>

In [ ]:
data = pd.read_csv('preprocessedtweetsfinal.csv')
corpus=data['tweet']
y=data['sentiment']



from sklearn.feature_extraction.text import TfidfVectorizer
cv = TfidfVectorizer(max_features=1500)
X = cv.fit_transform(corpus).toarray()
#y = datasset.iloc[:, 1].values

#splitting the dtaset into the trning set and testset
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20,random_state = 0)

from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier()
classifier.fit(X_train, y_train)

#predicting the test set results
y_pred = classifier.predict(X_test)

#mking confusion matrix
from sklearn.metrics import confusion_matrix
cm =  confusion_matrix(y_test, y_pred)
print(cm)
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_pred))